In [1]:
import os, json
import importlib
import torch
import torch_geometric

# deprecated in torch_geometric 2.0
try:
    from torch_geometric.loader import DataLoader
except:
    from torch_geometric.data import DataLoader

import hydragnn

import voronoi_utils as voronoi_utils
importlib.reload(voronoi_utils)

import torch_geometric.transforms as T

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import to_networkx
%matplotlib inline

import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42) 
#torch.use_deterministic_algorithms(True)
dataset = voronoi_utils.VoronoiNet(root='.',force_reload=True) # 

Processing...
Done!


In [2]:
perm = torch.tensor([319,  12,  81, 137, 351,  34, 103, 258, 322, 118, 307, 252, 284, 293,
        163, 291,  21, 201, 312, 187, 349, 246, 235, 203, 101,  83, 194, 129,
        133, 272, 158, 124, 317, 268,  37, 172,  14, 162, 105, 113, 279, 354,
          8, 393, 170,  52, 396,  86, 374, 225,   6, 372, 238,  44, 224,  19,
        320,  54, 237, 245, 115, 278, 242, 205, 303, 357,  41, 256, 298, 375,
        220,  65, 111,  94, 324, 292, 335, 161,  95, 380, 151, 202, 346, 231,
        149, 325, 333, 119,  42,  99, 106, 207, 355, 330, 184, 281, 146, 232,
          3, 369, 306, 261, 344, 358, 204, 387, 309, 211,  63, 222, 361, 301,
        260, 288, 345,  46,   9, 388, 155,  64, 153, 247, 191,  61,  36, 287,
        255, 141, 316, 348, 125, 399,  70, 311,  53,  38,  15, 304, 365,  32,
         10, 290, 336, 159,  79, 389, 223, 285,  20, 126, 264, 175, 193, 167,
        107, 171, 244, 189, 134,  91, 283, 179, 318, 230,  39,  25, 112,  59,
         11, 152, 249, 366, 228, 371, 182, 381, 394, 132, 385, 343, 271, 254,
         78, 305, 367, 282, 377, 362,  48, 120, 337, 160, 145,  97,  23,  77,
        199, 379,  87, 269, 263, 192, 378,  50, 323, 174, 350,  71,  93, 262,
        383, 259, 123,  85, 390, 135, 314, 164,  28, 190,  26, 294,  45, 173,
        197,   1, 110,  31, 215, 275, 327, 143, 116, 217, 154, 328, 196, 214,
         67, 286,   5,  96, 178, 165, 136, 313, 347,   7,  76, 157, 310, 382,
         74,  75,  90, 195, 156, 329, 338,  49,  68, 121, 176, 359, 188, 186,
        147,  92, 169,  47, 257, 168, 243, 297,  89, 138, 239,   2,  84, 114,
        332, 180, 397, 208,  66, 300, 128,  18, 229, 295, 226, 331,  29, 276,
        227, 251, 392, 117, 360,  55, 221,  17, 148, 363, 206,  40, 209, 341,
        100,  57, 364, 368, 108, 308,  27, 373, 386,  62, 102, 236,  98, 200,
        130, 342, 289, 166, 321,  16, 139, 270, 240, 280, 150, 127,  33,  22,
         82, 233, 198, 241,  73, 299, 250, 326,  88, 266, 109, 273,  56,  35,
         30, 177, 210, 356,  58, 185, 144, 140, 265,  80, 183,  51,   0,   4,
         69, 104, 181,  72, 218, 142, 395, 296, 253,  60, 340, 353, 398, 339,
         24, 219, 277, 315, 352, 370, 376, 391, 274, 234, 131, 248, 334, 216,
        122, 213,  43, 384, 212,  13, 302, 267])
shuffled_dataset = dataset.index_select(perm)

In [3]:
# Set this path for output.
try:
    os.environ["SERIALIZED_DATA_PATH"]
except:
    os.environ["SERIALIZED_DATA_PATH"] = os.getcwd()

# num_samples = 64

# Configurable run choices (JSON file that accompanies this example script).
#filename = os.path.join(os.path.dirname(os.path.abspath(__file__)), "qm9.json")

# filename = os.path.join(os.getcwd(), "voronoi_PNA.json")
filename = os.path.join(os.getcwd(), "voronoi_pna_activation_func.json")
with open(filename, "r") as f:
    config = json.load(f)
verbosity = config["Verbosity"]["level"]
var_config = config["NeuralNetwork"]["Variables_of_interest"]

# Always initialize for multi-rank training.
world_size, world_rank = hydragnn.utils.setup_ddp()

log_name = "activation_func/128x10_sigmoid"
# Enable print to log file.
hydragnn.utils.setup_log(log_name)

# Use built-in torch_geometric dataset.
# Filter function above used to run quick example.
# NOTE: data is moved to the device in the pre-transform.
# NOTE: transforms/filters will NOT be re-run unless the qm9/processed/ directory is removed.

# dataset = torch_geometric.datasets.QM9(
#     root="dataset/qm9", pre_transform=qm9_pre_transform, pre_filter=qm9_pre_filter
# )
train, val, test = hydragnn.preprocess.split_dataset(
    shuffled_dataset, config["NeuralNetwork"]["Training"]["perc_train"], False
)
(train_loader, val_loader, test_loader,) = hydragnn.preprocess.create_dataloaders(
    train, val, test, config["NeuralNetwork"]["Training"]["batch_size"]
)

config = hydragnn.utils.update_config(config, train_loader, val_loader, test_loader)

model = hydragnn.models.create_model_config(
    config=config["NeuralNetwork"],
    verbosity=verbosity,
)
model = hydragnn.utils.get_distributed_model(model, verbosity)

learning_rate = config["NeuralNetwork"]["Training"]["Optimizer"]["learning_rate"]
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=5, min_lr=0.000001
)

Distributed data parallel: nccl master at 127.0.0.1:8890


[W socket.cpp:464] [c10d] The server socket has failed to bind to [::]:8890 (errno: 98 - Address already in use).
[W socket.cpp:464] [c10d] The server socket has failed to bind to Hamilton.utk.tennessee.edu:8890 (errno: 98 - Address already in use).
[E socket.cpp:500] [c10d] The server socket has failed to listen on any local network address.


DistNetworkError: The server socket has failed to listen on any local network address. The server socket has failed to bind to [::]:8890 (errno: 98 - Address already in use). The server socket has failed to bind to Hamilton.utk.tennessee.edu:8890 (errno: 98 - Address already in use).

In [ ]:
os.getcwd()

'/home/ecalisk1/git/HydraGNN/examples/voronoi_composite/pna'

In [ ]:
# Run training with the given model and Voronoi dataset.
writer = hydragnn.utils.get_summary_writer(log_name)
hydragnn.utils.save_config(config, log_name)

hydragnn.train.train_validate_test(
    model,
    optimizer,
    train_loader,
    val_loader,
    test_loader,
    writer,
    scheduler,
    config["NeuralNetwork"],
    log_name,
    verbosity,
    create_plots=config["Visualization"]["create_plots"],
)

TypeError: 'NoneType' object is not callable

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w -r -m

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: Tue Jun 11 2024

Python implementation: CPython
Python version       : 3.12.2
IPython version      : 8.25.0

Compiler    : GCC 12.3.0
OS          : Linux
Release     : 5.15.146.1-microsoft-standard-WSL2
Machine     : x86_64
Processor   : x86_64
CPU cores   : 56
Architecture: 64bit

Git repo: git@github.com:erdemcaliskan/HydraGNN.git

matplotlib     : 3.8.4
torch          : 2.2.2
torch_geometric: 2.5.2
json           : 2.0.9
hydragnn       : 3.0rc1
networkx       : 3.1
numpy          : 1.26.4

Watermark: 2.4.3



In [ ]:
import torch

idx1 = torch.tensor([[1], [2], [2]])
print(idx1)
idx2 = torch.tensor([[1, 2, 2]]).t()
print(idx2)
print(idx1 ==idx2)
# idx1 and idx2 are the same but the output results are different. Why?

a = torch.tensor([[0.6280, 0.5672, 0.3760],
                  [0.4340, 0.9902, 0.1539],
                  [0.4585, 0.2638, 0.6983]])

print(a.gather(0, idx1))
print(a.gather(0, idx2))
%watermark -p torch

tensor([[1],
        [2],
        [2]])
tensor([[1],
        [2],
        [2]])
tensor([[True],
        [True],
        [True]])
tensor([[0.4340],
        [0.4585],
        [0.4585]])
tensor([[0.4340],
        [0.4585],
        [0.4585]])
torch: 2.2.2

